In [96]:
import pandas as pd
import numpy as np

In [97]:
schools_path = "Resources/schools_complete.csv"
schoolsdf = pd.read_csv(schools_path)

In [98]:
students_path = "Resources/students_complete.csv"
studentsdf = pd.read_csv(students_path)

In [99]:
#combine data 
SchoolTotaldf = pd.merge(studentsdf,schoolsdf, on="school_name", how="left")

## District Summary

In [100]:
#Calculate the total number of schools
totalschools = len(SchoolTotaldf["School ID"].unique())

In [101]:
#Calculate the total number of students
totalstudents = len(SchoolTotaldf["Student ID"].unique())

In [102]:
#Calculate the total budget
totalbudget = schoolsdf["budget"].sum()

In [103]:
#Calculate the average math score
mathaverage = SchoolTotaldf["math_score"].mean()

In [104]:
#Calculate the average reading score
readaverage = SchoolTotaldf["reading_score"].mean()

In [105]:
#Calculate the percentage of students with a passing math score (70 or greater)
    #isolate the student with >= passmark
mathpassdf = SchoolTotaldf.loc[SchoolTotaldf["math_score"]>=70, :]
    
    #count the number of students with pass score
mathpasscount = mathpassdf["Student ID"].count()
    #percentage calculation
masthpasspercent = mathpasscount/totalstudents

In [106]:
#Calculate the percentage of students with a passing reading score (70 or greater)

readpassdf = SchoolTotaldf.loc[SchoolTotaldf["reading_score"]>=70,:]

readpasscount = readpassdf["Student ID"].count()

readpasspercent = readpasscount/totalstudents

In [107]:
#Calculate the percentage of students who passed math and reading (% Overall Passing)

# sample code: df.loc[(df["B"] > 50) & (df["C"] == 900), "A"]
bothpassdf = SchoolTotaldf.loc[(SchoolTotaldf["math_score"]>=70) & (SchoolTotaldf["reading_score"]>=70), :]

bothpasscount = bothpassdf["Student ID"].count()

bothpasspercent = bothpasscount/totalstudents

In [108]:
#Create a dataframe to hold the above results

DistrictSummary = pd.DataFrame({
    "Total Schools":[totalschools],
    "Total Students":[totalstudents],
    "Total Budget":[totalbudget],
    "Average Math Score":[mathaverage],
    "Average Reading Score":[readaverage],
    "% Passing Math":[masthpasspercent],
    "% Passing Reading":[readpasspercent],
    "% Overall Passing":[bothpasspercent],
    })


In [109]:
#Give the displayed data cleaner formatting
DistrictSummary["Total Students"] = DistrictSummary["Total Students"].map("{:,}".format)
DistrictSummary["Total Budget"] = DistrictSummary["Total Budget"].map("${:,.2f}".format)
DistrictSummary["Average Math Score"] = DistrictSummary["Average Math Score"].map("{:.2f}".format)
DistrictSummary["Average Reading Score"] = DistrictSummary["Average Reading Score"].map("{:.2f}".format)
DistrictSummary["% Passing Math"] = DistrictSummary["% Passing Math"].map("{:.2%}".format)
DistrictSummary["% Passing Reading"] = DistrictSummary["% Passing Reading"].map("{:.2%}".format)
DistrictSummary["% Overall Passing"] = DistrictSummary["% Overall Passing"].map("{:.2%}".format)

DistrictSummary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


## School Summary

In [110]:
#Starting table
SchoolSummary = schoolsdf

In [111]:
#  Add Per Student Budget
SchoolSummary["Per Student Budget"] = SchoolSummary["budget"]/SchoolSummary["size"]

In [112]:
#Get the means of reading and math score
schoolgroupmeans = studentsdf.groupby("school_name").agg({'reading_score' : 'mean', 'math_score' : 'mean'})

In [113]:
schoolgroupmeans = schoolgroupmeans.rename(columns={"reading_score":"Average Reading Score", "math_score":"Average Math Score"})

In [114]:
#  Add the % Passing Math (The percentage of students that passed math.)
    #Can use existing dataframe mathpassdf that already collects students with pass mark

mathpassschool = pd.DataFrame(mathpassdf.groupby("school_name")["math_score"].count())

In [115]:
mathpassschool = mathpassschool.rename(columns={"math_score":"Passing Math Count"})

In [116]:
#  Add the % Passing Read
    #use existing dataframe readpassdf
readpassschool = pd.DataFrame(readpassdf.groupby("school_name")["reading_score"].count())

In [117]:
readpassschool = readpassschool.rename(columns={"reading_score":"Passing Reading Count"})

In [118]:
#  Add the % Passing both
    #use existing dataframe bothpassdf
bothpassschool = bothpassdf.groupby("school_name").agg({'reading_score' : 'count', 'math_score' : 'count'})

In [119]:
#same count each column so only need one column
bothpassschool = bothpassschool.rename(columns={"reading_score":"Overall Passing Count"})

In [120]:
#bothpassschool = bothpassschool.drop(["math_score"], axis = 1)
#bothpassschool

In [121]:
#join all tables together
    #SchoolSummary
    #schoolgroupmeans
    #mathpassschool
    #readpassschool
    #bothpassschool
    #sample code: pd.merge(pd.merge(df1,df2,on='name'),df3,on='name')
        #code was long so entered at practical places. Indent was automatic and if I altered code wuold no work

merge = pd.merge(pd.merge
                 (pd.merge
                          (pd.merge(SchoolSummary,schoolgroupmeans, on="school_name"), 
                                   mathpassschool, on="school_name"), 
                          readpassschool, on="school_name"),
                 bothpassschool, on="school_name") 


In [122]:
dropschoolID = merge.drop(["School ID"], axis = 1)

In [123]:
reindex = dropschoolID.set_index("school_name")

In [124]:
#Change Pass Count columns to percentage figures

reindex["Passing Math Count"] = reindex["Passing Math Count"]/reindex["size"]
reindex["Passing Reading Count"] = reindex["Passing Reading Count"]/reindex["size"]
reindex["Overall Passing Count"] = reindex["Overall Passing Count"]/reindex["size"]

In [125]:
#rename columns
reindex = reindex.rename(columns={
                                  "type":"School Type",
                                  "size":"Total Students",
                                  "budget":"Total School Budget",
                                  "Passing Math Count":"% Passing Math",
                                  "Passing Reading Count":"% Passing Reading",
                                  "Overall Passing Count":"% Overall Passing",
                                  })

#reorder columns
reindex = reindex[[
    "School Type",
    "Total Students",
    "Total School Budget",
    "Per Student Budget",
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing"
]]

#create copy as need to use orginal later
reindexCopy = reindex.copy()

#format

reindexCopy["Total School Budget"] = reindexCopy["Total School Budget"].map("${:,.2f}".format)
reindexCopy["Per Student Budget"] = reindexCopy["Per Student Budget"].map("${:,.2f}".format)
reindexCopy["Average Math Score"] = reindexCopy["Average Math Score"].map("{:.2f}".format)
reindexCopy["Average Reading Score"] = reindexCopy["Average Reading Score"].map("{:.2f}".format)
reindexCopy["% Passing Math"] = reindexCopy["% Passing Math"].map("{:.2%}".format)
reindexCopy["% Passing Reading"] = reindexCopy["% Passing Reading"].map("{:.2%}".format)
reindexCopy["% Overall Passing"] = reindexCopy["% Overall Passing"].map("{:.2%}".format)

#Sort index
SchoolSummaryTable = reindexCopy.sort_index(ascending=True)

SchoolSummaryTable

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,54.64%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,54.29%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,89.23%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Top Performing Schools (By % Overall Passing)

In [126]:
TopFiveSchools = SchoolSummaryTable.sort_values(by=['% Overall Passing'], ascending=False)
TopFiveSchools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,91.33%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,90.95%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,90.60%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,90.58%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,90.54%


## Bottom Performing Schools (By % Overall Passing)

In [127]:
BottomFiveSchools = SchoolSummaryTable.sort_values(by=['% Overall Passing'], ascending=True)
BottomFiveSchools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,52.99%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,53.20%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,53.51%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,53.53%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,53.54%


## Math and Reading Scores by Grade

In [128]:
#Create a table that lists the average Math Score for students of each grade level
    
    #Collect 9th grades and calculate average
averageNine = SchoolTotaldf.loc[SchoolTotaldf["grade"]=="9th", :]
averageNinegroup = averageNine.groupby(["school_name"]).mean()

    #separate math score from reading score
Mathnine = averageNinegroup[["math_score"]]
Mathnine = Mathnine.rename(columns={"math_score":"9th"})

Readnine = averageNinegroup[["reading_score"]]
Readnine = Readnine.rename(columns={"reading_score":"9th"})

In [129]:
averageTen = SchoolTotaldf.loc[SchoolTotaldf["grade"]=="10th", :]
averageTengroup = averageTen.groupby(["school_name"]).mean()

MathTen = averageTengroup[["math_score"]]
MathTen = MathTen.rename(columns={"math_score":"10th"})

ReadTen = averageTengroup[["reading_score"]]
ReadTen = ReadTen.rename(columns={"reading_score":"10th"})

In [130]:
averageElev = SchoolTotaldf.loc[SchoolTotaldf["grade"]=="11th", :]
averageElevgroup = averageElev.groupby(["school_name"]).mean()

MathElev = averageElevgroup[["math_score"]]
MathElev = MathElev.rename(columns={"math_score":"11th"})

ReadElev = averageElevgroup[["reading_score"]]
ReadElev = ReadElev.rename(columns={"reading_score":"11th"})

In [131]:
averageTwel = SchoolTotaldf.loc[SchoolTotaldf["grade"]=="12th", :]
averageTwelgroup = averageTwel.groupby(["school_name"]).mean()

MathTwel = averageTwelgroup[["math_score"]]
MathTwel = MathTwel.rename(columns={"math_score":"12th"})

ReadTwel = averageTwelgroup[["reading_score"]]
ReadTwel = ReadTwel.rename(columns={"reading_score":"12th"})

In [132]:
#Merge all math grade columns
MathScoresbyGrade = pd.merge(
    pd.merge(pd.merge(
        Mathnine,MathTen, on="school_name"), 
             MathElev, on="school_name"),
    MathTwel, on="school_name")
MathScoresbyGrade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [133]:
#Merge all reading grade columns
ReadingScoresbyGrade = pd.merge(
    pd.merge(pd.merge(
        Readnine,ReadTen, on="school_name"), 
             ReadElev, on="school_name"),
    ReadTwel, on="school_name")
ReadingScoresbyGrade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

In [134]:
# Used code to calculate bins - pd.cut(reindex["Per Student Budget"], bins=4) - but result not useful
        #Used bin values in starter notebook

bins = [0, 585, 630, 645, 676]
group_names = ["< $584", "$585-629", "$630-644", "$645-675"]

In [135]:
#make copy of source table reindex
spendbins = reindex.copy()

#impliment bins into table
spendbins["Spending Ranges (Per Student)"] = pd.cut(spendbins["Per Student Budget"], bins, labels=group_names, include_lowest = True)

In [136]:
#Convert ranges to index
spendbinsgroup = spendbins.groupby(["Spending Ranges (Per Student)"])

spendbinsmean = spendbinsgroup.mean()

#Organise columns
spendbinsmean = spendbinsmean[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]

#Copy to protect original dataframe
ScoresbySchoolSpending = spendbinsmean.copy()

#Format
ScoresbySchoolSpending["Average Math Score"] = ScoresbySchoolSpending["Average Math Score"].map("{:.2f}".format)
ScoresbySchoolSpending["Average Reading Score"] = ScoresbySchoolSpending["Average Reading Score"].map("{:.2f}".format)
ScoresbySchoolSpending["% Passing Math"] = ScoresbySchoolSpending["% Passing Math"].map("{:.2%}".format)
ScoresbySchoolSpending["% Passing Reading"] = ScoresbySchoolSpending["% Passing Reading"].map("{:.2%}".format)
ScoresbySchoolSpending["% Overall Passing"] = ScoresbySchoolSpending["% Overall Passing"].map("{:.2%}".format)

ScoresbySchoolSpending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
< $584,83.46,83.93,93.46%,96.61%,90.37%
$585-629,81.90,83.16,87.13%,92.72%,81.42%
$630-644,78.52,81.62,73.48%,84.39%,62.86%
$645-675,77.00,81.03,66.16%,81.13%,53.53%


## Scores by School Size

In [137]:
sizebins = [0, 1000, 2001, 5001]
size_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [138]:
#make copy of source table SchoolSummaryTable
addsizebins = reindex.copy()

#implement bins
addsizebins["School Size"] = pd.cut(addsizebins["Total Students"], sizebins, labels=size_names, include_lowest = True)

In [139]:
#convert sizebins to index
sizebinsgroup = addsizebins.groupby(["School Size"])

sizebinsmean = sizebinsgroup.mean()

sizebinsmean = sizebinsmean[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]
ScoresbySchoolSize = sizebinsmean.copy()

ScoresbySchoolSize["Average Math Score"] = ScoresbySchoolSize["Average Math Score"].map("{:.2f}".format)
ScoresbySchoolSize["Average Reading Score"] = ScoresbySchoolSize["Average Reading Score"].map("{:.2f}".format)
ScoresbySchoolSize["% Passing Math"] = ScoresbySchoolSize["% Passing Math"].map("{:.2%}".format)
ScoresbySchoolSize["% Passing Reading"] = ScoresbySchoolSize["% Passing Reading"].map("{:.2%}".format)
ScoresbySchoolSize["% Overall Passing"] = ScoresbySchoolSize["% Overall Passing"].map("{:.2%}".format)

ScoresbySchoolSize

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,89.88%
Medium (1000-2000),83.37,83.86,93.60%,96.79%,90.62%
Large (2000-5000),77.75,81.34,69.96%,82.77%,58.29%


## Scores by School Type

In [140]:
#make copy of source table SchoolSummaryTable
schooltypes = reindex.copy()

In [141]:
#set index to school types
typesgroup = schooltypes.groupby(["School Type"])

typesgroupmean = typesgroup.mean()

typesgroupmean = typesgroupmean[["Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]
ScoresbySchoolType = typesgroupmean.copy()

ScoresbySchoolType["Average Math Score"] = ScoresbySchoolType["Average Math Score"].map("{:.2f}".format)
ScoresbySchoolType["Average Reading Score"] = ScoresbySchoolType["Average Reading Score"].map("{:.2f}".format)
ScoresbySchoolType["% Passing Math"] = ScoresbySchoolType["% Passing Math"].map("{:.2%}".format)
ScoresbySchoolType["% Passing Reading"] = ScoresbySchoolType["% Passing Reading"].map("{:.2%}".format)
ScoresbySchoolType["% Overall Passing"] = ScoresbySchoolType["% Overall Passing"].map("{:.2%}".format)

ScoresbySchoolType

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%


## Pandas Challenge: Observed trends

* The “Scores by School Type” summary chart indicates that Charter schools have a substantially higher overall passing rate that district schools. The main contributing factor being district schools having a lower math passing rate.

*  The “Scores by School Size” summary chart also shows that large schools (over 2000 students) have low overall pass marks, with maths pass rates falling well below small and medium size schools.

* Both the “Top” and “Bottom Five Performing School” charts again highlight the difference between charter and district schools. All the top schools are charter schools, with lower school sizes that district schools, which are all the school types in the bottom five.  It’s clear that the size of a school impacts the overall passing rate of that school, with charter schools' ability to keep down sizes being an advantage to students.

* Interestingly, spending more per student does not impact the passing rate, seen by the “Scores by School Spending” chart. In fact, it shows that the schools with the higher spending per student are achieving the lowest pass marks. It could indicate that district schools are very inefficient with large schools sizes with large per student spends and the lowest passing rates.

